# Práctica Chromabd

In [1]:
# 1️⃣ Importar librerías
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import os
import numpy as np
import uuid

# Modelo de embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

def get_embeddings(text):
    return model.encode(text).tolist()

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="documentos_ia")

def cargar_documentos():
    docs_path = "docs/"
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

# --- CRUD FUNCIONES ---

def create_example(new_doc):
    """Añadir un documento nuevo"""
    embedding = get_embeddings(new_doc)
    doc_id = str(uuid.uuid4())
    metadata = {"source": "nuevo_documento.txt"}
    
    collection.add(
        documents=[new_doc],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"✅ Documento añadido con ID: {doc_id}")

def read_example(query):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=1
    )
    
    if results["documents"]:
        print(f"📄 Mejor coincidencia: {results['documents'][0][0]}")
        print(f"📁 Fuente: {results['metadatas'][0][0]['source']}")
        print(f"🆔 ID: {results['ids'][0][0]}")
    else:
        print("⚠️ No se encontraron resultados.")

def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    metadata = {"source": "documento_actualizado.txt"}
    
    collection.add(
        documents=[new_text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[new_id]
    )
    print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")

def delete_example(doc_id):
    """Eliminar por ID"""
    collection.delete(ids=[doc_id])
    print(f"🗑️ Documento con ID '{doc_id}' eliminado.")

c:\Proyectos\UBA\VectorDatabases\basesdatosIA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
crear_base_datos()

✅ Base de datos creada con éxito.


In [3]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [4]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: cce8b633-0e76-4958-a2b2-f7d471579083, Contenido: Había una vez, en un reino lejano, una joven llamada Cenicienta, quien vivía con su madrastra y herm  Metadata: {'source': 'doc1.txt'} Embedding: [0.03695862367749214, 0.09123437851667404, -0.006070955656468868, -0.12097117304801941, -0.08920404314994812]...
ID: 7a719259-97b1-4c4f-8ae5-dde12cae83a4, Contenido: siosas por impresionar al príncipe, se prepararon con lujosos vestidos y joyas, pero dejaron a Cenic  Metadata: {'source': 'doc1.txt'} Embedding: [0.006394681986421347, 0.08617572486400604, -0.0019227326847612858, -0.05962548404932022, -0.061944372951984406]...
ID: ed7b7a7e-8dd1-424c-91e7-2116ac773acf, Contenido: a huyó dejando atrás uno de sus zapatos de cristal.

El príncipe, decidido a encontrarla, recorrió e  Metadata: {'source': 'doc1.txt'} Embedding: [-0.03151360899209976, 0.08306147158145905, -0.07771097123622894, -0.01462840661406517, -0.08390384167432785]...
ID: eba282bb-919b-45d6-b22b-8cc323f4da24, Contenido: Había un

## Ejercicio 1: Crear y Consultar 
Crea un documento nuevo con el texto: "La zorra y el gato iban caminando juntos por el bosque, y comenzaron a hablar sobre lo que harían si los atacaban los perros de caza. La zorra se jactaba de tener muchas estrategias, mientras que el gato decía que solo conocía una: trepar a un árbol para escapar". 
Realiza una consulta para encontrar documentos similares a la frase: "Cuando hay peligro, algunos animales prefieren escapar trepando antes que confiar en planes complicados".

In [5]:
new_doc = "Había una vez un lobo malvado que asustaba a todos los animales del bosque. Un día encontró a una astuta zorra que lo engañó para salvarse."

create_example(new_doc)

✅ Documento añadido con ID: cff757f7-c547-43e2-9cdd-2c946ec0d821


In [6]:
query = "Un lobo engañoso y una zorra astuta en el bosque."

read_example(query)

📄 Mejor coincidencia: Había una vez un lobo malvado que asustaba a todos los animales del bosque. Un día encontró a una astuta zorra que lo engañó para salvarse.
📁 Fuente: nuevo_documento.txt
🆔 ID: cff757f7-c547-43e2-9cdd-2c946ec0d821


### ¿Qué tipo de operación CRUD usaste primero?

La primera operación utilizada fue la "C" de Create.

### ¿Cuál es el ID del documento creado?

Documento añadido con ID: cff757f7-c547-43e2-9cdd-2c946ec0d821

### Explica por qué el documento aparece como coincidencia en la consulta.

La consulta fue:

> "Cuando hay peligro, algunos animales prefieren escapar trepando antes que confiar en planes complicados."

El documento aparece como la mejor coincidencia en la consulta porque el texto de la consulta ("Un lobo engañoso y una zorra astuta en el bosque") es muy similar al contenido del documento creado, que habla del lobo y la zorra en el bosque. La similitud semántica entre las palabras clave ("lobo", "zorra", "bosque", "engañó") y el contexto general hace que el sistema lo considere como una coincidencia relevante.


## Ejercicio 2: Actualizar y Eliminar 
Crea un documento con el texto: "Había una vez un lobo malvado que asustaba a todos los animales del bosque." 

Encuentra su ID, luego actualízalo por: "Un lobo astuto engañó a los animales del bosque para conseguir lo que quería."

Elimina el documento original usando el mismo ID. 

In [7]:
new_doc2 = "Había una vez un lobo malvado que asustaba a todos los animales del bosque."

create_example(new_doc2)

✅ Documento añadido con ID: 46f6ce7f-c9af-4340-b7e3-e601d505b4ec


In [8]:
new_text = "Un lobo astuto engañó a los animales del bosque para conseguir lo que quería."
old_id = "46f6ce7f-c9af-4340-b7e3-e601d505b4ec"

update_example(old_id, new_text)

♻️ Documento actualizado. Nuevo ID: 9d94e31f-d9f7-41d3-8210-bef6376fa077


### ¿Qué ocurre si intentas actualizar un documento eliminado? 

In [9]:
update_example(old_id, new_text)

Delete of nonexisting embedding ID: 46f6ce7f-c9af-4340-b7e3-e601d505b4ec
Delete of nonexisting embedding ID: 46f6ce7f-c9af-4340-b7e3-e601d505b4ec


♻️ Documento actualizado. Nuevo ID: 0ea43c9d-05bf-4a74-b1f4-fcdecc27f6d2


En la función `update_example`, la operación de actualización consiste en **eliminar un documento por su ID** y luego **insertar uno nuevo con contenido actualizado**.

Si intentás actualizar un documento que **ya fue eliminado anteriormente**, la operación `collection.delete(ids=[old_id])` simplemente **no encuentra nada que eliminar**, y **no genera un error crítico**. Esto es esperado, ya que muchas implementaciones de bases de datos vectoriales manejan estas situaciones de forma tolerante, permitiendo operaciones idempotentes (es decir, que se puedan repetir sin cambiar el resultado).

### ¿Cómo se garantiza la integridad de los datos en las operaciones? 

La integridad de los datos se garantiza mediante:

1. **IDs únicos:** Cada documento tiene un `id` único (en este caso generado con `uuid.uuid4()`), lo que evita colisiones o sobrescrituras accidentales.
2. **Eliminación explícita:** Al eliminar antes de insertar, evitamos que haya duplicados no deseados del mismo contenido modificado.
3. **Control de metadatos:** Al usar campos como `"source"` dentro de `metadata`, se puede rastrear el origen y estado del documento, incluso después de ser actualizado.
4. **Uso de embeddings consistentes:** Cada nuevo documento se vectoriza al momento de ser insertado, lo que asegura que su representación semántica esté alineada con su contenido actual.

Este enfoque de "eliminar y reinsertar" es simple pero efectivo para garantizar consistencia en bases de datos vectoriales que no siempre permiten operaciones `update` nativas.

## Ejercicio 3: Errores y Validación 
Intenta eliminar un documento con el ID "no_existe". 

Crea un nuevo documento sin usar get_embeddings() (ejemplo: enviar un array vacío). 

In [10]:
delete_example("no_existe")

Delete of nonexisting embedding ID: no_existe
Delete of nonexisting embedding ID: no_existe


🗑️ Documento con ID 'no_existe' eliminado.


In [ ]:
# Crear un documento sin usar get_embeddings()
collection.add(
    documents=["Texto sin embedding"],
    embeddings=[[]],  # array vacío
    metadatas=[{"source": "sin_embedding.txt"}],
    ids=[str(uuid.uuid4())]
)

--- Logging error ---
Traceback (most recent call last):
  File "c:\Proyectos\UBA\VectorDatabases\basesdatosIA\Lib\site-packages\chromadb\db\mixins\embeddings_queue.py", line 308, in _notify_one
    sub.callback(filtered_embeddings)
  File "c:\Proyectos\UBA\VectorDatabases\basesdatosIA\Lib\site-packages\chromadb\segment\impl\vector\local_persistent_hnsw.py", line 211, in _write_records
    self._ensure_index(len(records), len(record["embedding"]))
  File "c:\Proyectos\UBA\VectorDatabases\basesdatosIA\Lib\site-packages\chromadb\segment\impl\vector\local_hnsw.py", line 208, in _ensure_index
    raise InvalidDimensionException(
chromadb.errors.InvalidDimensionException: Dimensionality of (0) does not match indexdimensionality (384)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\caro_\.pyenv\pyenv-win\versions\3.11.9\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^

### ¿Qué errores ocurren en cada caso? 

- **Eliminar un documento con un ID inexistente:**

  No se genera un error crítico. ChromaDB simplemente ignora el ID si no existe en la colección. Este comportamiento es **idempotente y seguro**, lo que permite hacer múltiples intentos de eliminación sin afectar la integridad de la base.

- **Agregar un documento sin `get_embeddings()` (por ejemplo, con una lista vacía):**

  Se genera un **`ValueError` o `RuntimeError`**, dependiendo del backend utilizado. Esto ocurre porque el sistema espera un **vector numérico válido**, y un array vacío no cumple con ese requisito. La operación falla al intentar procesar el embedding inválido.

### ¿Cómo prevenir estos problemas? 

- Para evitar errores al eliminar documentos, se recomienda verificar previamente si el ID existe en la colección. Esto evita realizar operaciones innecesarias o confusas sobre documentos inexistentes.

- Para prevenir fallos al agregar documentos, es fundamental asegurarse de que los embeddings se hayan generado correctamente y que no estén vacíos. Validar este paso garantiza que el sistema reciba datos en el formato esperado y evita errores en tiempo de ejecución.

## Ejercicio 4: Consultas por Tema 
Crea tres documentos: 
* "La madrastra malvada de Blanca Nieves y su plan oculto" 
* "La bruja de Hansel y Gretel y su trampa mortal" 
* "El lobo de Caperucita Roja y su engaño malévolo" 

Realiza una consulta por la frase: "Villanos de los cuentos de los Hermanos Grimm". 

In [14]:
doc1 = "La madrastra malvada de Blanca Nieves y su plan oculto" 
doc2 = "La bruja de Hansel y Gretel y su trampa mortal" 
doc3 = "El lobo de Caperucita Roja y su engaño malévolo" 

create_example(doc1)
create_example(doc2)
create_example(doc3)

✅ Documento añadido con ID: 2d5fc8ad-8d87-4bc9-bb05-c229bb676ca2
✅ Documento añadido con ID: 096a7602-356f-4b92-b0bf-1bf7e98fa9b5
✅ Documento añadido con ID: 5487cf39-ea1c-4975-bfce-03a9106372f1


In [15]:
consulta = "Villanos de los cuentos de los Hermanos Grimm"
read_example(consulta)

📄 Mejor coincidencia: La bruja de Hansel y Gretel y su trampa mortal
📁 Fuente: nuevo_documento.txt
🆔 ID: 096a7602-356f-4b92-b0bf-1bf7e98fa9b5


### ¿Cuáles documentos aparecen como coincidencias? 

**"La bruja de Hansel y Gretel y su trampa mortal"** fue identificado como la mejor coincidencia, lo que tiene sentido, dado que la consulta está enfocada en villanos, y la bruja es uno de los villanos más emblemáticos de los cuentos de los Hermanos Grimm.

### Analiza cómo las palabras clave influyen en los resultados.

La razón de esta coincidencia puede ser que las palabras clave de la consulta "villanos", "trampa", y "mortal" están estrechamente relacionadas con el contexto de la bruja en la historia, que es conocida por su papel de antagonista.

Los otros villanos como la madrastra de Blanca Nieves y el lobo de Caperucita Roja no coincidieron tan bien probablemente debido a que sus descripciones o palabras clave no se ajustaron tan estrechamente a las especificaciones de la consulta.

## Ejercicio 5: Eliminaciones Selectivas 
Crea dos documentos con tópicos opuestos (ejemplo: uno sobre "Ecología" y otro sobre 
"Minería de datos"). 

Elimina el documento sobre ecología usando su ID. 

Verifica que solo quede un documento. 

¿Cómo usar collection.count() para confirmar? 


Usaremos los conceptos de "magia" y "realidad" como temas opuestos dentro de los cuentos. Uno puede hablar sobre "magia" (como algo fantástico y sobrenatural), y el otro sobre "realidad" (como los aspectos más mundanos y reales de la vida cotidiana).

In [21]:
create_example("La magia esta presente en los cuentos")
create_example("Los personajes de los cuentos viven una realida muy dura")

✅ Documento añadido con ID: 1110d7b5-6c6c-45cf-aff6-153795d5f196
✅ Documento añadido con ID: 5fe5741d-986b-4c3f-a29a-8692be3f8072


In [22]:
count = collection.count()
print(f"Cantidad de documentos en la colección: {count}")

Cantidad de documentos en la colección: 21


In [23]:
# Se eliminara el documento relacionado con la magia.

id_magia = "1110d7b5-6c6c-45cf-aff6-153795d5f196"
delete_example(id_magia)

🗑️ Documento con ID '1110d7b5-6c6c-45cf-aff6-153795d5f196' eliminado.


In [24]:
count = collection.count()
print(f"Cantidad de documentos restantes en la colección: {count}")

Cantidad de documentos restantes en la colección: 20


## Ejercicio 6: Actualización en Cadena 

Crea un documento con el texto "Los dilemas morales en los cuentos de los Hermanos Grimm". 

Actualízalo dos veces: 
* Primera actualización: "El sacrificio de los niños en Hansel y Gretel y el dilema de la supervivencia" 
* Segunda actualización: "Las consecuencias de las decisiones de los villanos en los cuentos de Cenicienta y Blanca Nieves". 

In [31]:
create_example("Los dilemas morales en los cuentos de los Hermanos Grimm")

✅ Documento añadido con ID: a30a9c40-05ba-4004-8ce0-2bcfcaf3ea39


In [32]:
id_doc_inicial = "a30a9c40-05ba-4004-8ce0-2bcfcaf3ea39"
update_example(id_doc_inicial, "El sacrificio de los niños en Hansel y Gretel y el dilema de la supervivencia")


♻️ Documento actualizado. Nuevo ID: 5676225e-b563-4c56-a38e-0023b5524829


In [33]:
# Segunda actualización: hablar sobre las consecuencias de las decisiones morales en los cuentos
update_example(id_doc_inicial, "Las consecuencias de las decisiones de los villanos en los cuentos de Cenicienta y Blanca Nieves")


Delete of nonexisting embedding ID: a30a9c40-05ba-4004-8ce0-2bcfcaf3ea39
Delete of nonexisting embedding ID: a30a9c40-05ba-4004-8ce0-2bcfcaf3ea39


♻️ Documento actualizado. Nuevo ID: a4a246e3-a59d-40a9-a173-788404eac98e


### ¿Cómo sigue evolucionando el contenido del documento con cada update? 

El documento empieza con una visión general de los dilemas morales en los cuentos, luego se especializa en un caso específico y, finalmente, aborda las consecuencias morales a lo largo de varios cuentos.

Este proceso refleja cómo el contenido del documento se actualiza en cadena, pasando de un tema general a uno más detallado y específico, para finalmente cubrir un conjunto más amplio de ejemplos.

## Ejercicio 7: Consultas Ambiguas 
Crea dos documentos similares pero no idénticos: 
* "Blanca Nieves era la más hermosa del reino" 
* "La reina preguntó quién era la más hermosa". 

Realiza una consulta por "La más hermosa". 

In [41]:
create_example("Blanca Nieves era la más hermosa del reino")
create_example("La reina preguntó quién era la más hermosa")

✅ Documento añadido con ID: 9adaf7fa-15f9-48e5-b22a-a8888e19b571
✅ Documento añadido con ID: aa5bc130-1726-4a2e-bac0-bd80b77e3943


In [42]:
read_example("La más hermosa")

📄 Mejor coincidencia: La reina preguntó quién era la más hermosa
📁 Fuente: nuevo_documento.txt
🆔 ID: aa5bc130-1726-4a2e-bac0-bd80b77e3943


### ¿Ambos documentos aparecen? 

No, solamente aparecio el 2er texto guardado. Eso indica que, ChromaDB identificó mayor similitud semántica con ese documento que con el otro.

### Explica la similitud semántica. 

Ambas frases comparten el mismo concepto central: el reconocimiento de quién es la más hermosa dentro del reino.
Sin embargo, están formuladas de manera diferente:

* Una es una afirmación directa sobre Blanca Nieves.

* La otra es una pregunta indirecta relacionada con la reina.

Aunque no hay coincidencia textual exacta entre la consulta "La más hermosa" y la frase completa "La reina preguntó quién era la más hermosa", el modelo pudo reconocer que ambas tratan el mismo significado.

El hecho de que ese segundo documento haya sido la mejor coincidencia, indica que el motor valoró más el contexto semántico en que aparece la expresión "la más hermosa" (dentro de una estructura narrativa).

## Ejercicio 8: Reiniciar Base de Datos 
Elimina todos los documentos usando delete() con el parámetro correcto. 

Vuelve a cargar la base desde cero con crear_base_datos(). 

¿Cuántos documentos hay ahora? (Usa collection.count()). 

In [ ]:
count = collection.count()
print(f"Cantidad de documentos en la colección: {count}")

Cantidad de documentos en la colección: 24


In [67]:
collection.delete(where={})

In [45]:
crear_base_datos()

✅ Base de datos creada con éxito.


In [46]:
count = collection.count()
print(f"Cantidad de documentos en la colección: {count}")

Cantidad de documentos en la colección: 9


## Ejercicio 9: Errores de Inserción 
Intenta crear un documento sin texto (ejemplo: cadena vacía). 

Corrige el error usando la función get_embeddings().

In [53]:
create_example("")


✅ Documento añadido con ID: 3dc7a66d-d7e8-4c3f-a5cc-9dbdc4d6308d


In [54]:
get_documents_by_id("3dc7a66d-d7e8-4c3f-a5cc-9dbdc4d6308d")

📄 Documento: 
📁 Fuente: nuevo_documento.txt


a función create_example() permite añadir nuevos documentos a la base de datos utilizando un modelo de embeddings. Sin embargo, en su versión actual, no contiene ninguna validación que impida insertar cadenas vacías.

Esto representa un problema, ya que:

* Una cadena vacía no aporta significado semántico.

* Aunque el modelo de embeddings pueda generar un vector (por ejemplo, uno por defecto o casi nulo), este no representa información útil.

* Se corre el riesgo de contaminar la base de datos con documentos irrelevantes o sin contenido.

Para solucionar esto se debería agregar una validación que verifique que el texto no esté vacío antes de generar el embedding e insertarlo. 

In [55]:
def create_example(new_doc):
    """Añadir un documento nuevo (con validación de contenido)"""
    if not new_doc.strip():
        print("❌ Error: No se puede insertar un documento vacío.")
        return

    embedding = get_embeddings(new_doc)
    doc_id = str(uuid.uuid4())
    metadata = {"source": "nuevo_documento.txt"}

    collection.add(
        documents=[new_doc],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"✅ Documento añadido con ID: {doc_id}")

In [56]:
create_example("")

❌ Error: No se puede insertar un documento vacío.


### ¿Qué lección aprendemos sobre los requisitos de ChromaDB?

* Es responsabilidad del desarrollador validar el contenido de los documentos antes de insertarlos.

* ChromaDB y los modelos de embeddings no siempre detendrán una inserción inválida, especialmente si se proporciona un embedding numéricamente válido.

* Incluir controles de calidad y limpieza de datos mejora la consistencia y utilidad de la base vectorial.

## Ejercicio opcional: Retrieval-Augmented Generation 

Intente levantar un llm en un endpoint local con ollama o LMStudio. Utilice el llm para generar respuestas aumentadas con contexto similar a una consulta dada. 

## ¿Cómo sigue esto?

### de forma conceptual debemos:
1. Recuperar el contexto relevante con una busqueda como read_example(query) o similar.
2. Formular el prompt para el llm, juntando la consulta original con los embeddings relevantes.
3. Enviar la Petición al Endpoint (en este caso LM Studio).
4. Recibir y mostrar la respuesta.

In [68]:
# Esta sección es para la integración con LM Studio y no es parte del CRUD. Es solo un ejemplo de cómo se podría hacer una consulta a un modelo de lenguaje como Llama 3 o similar.
# 2️⃣ Integración con LM Studio (Llama 3 o similar)
import requests
import json

LM_STUDIO_URL = "http://127.0.0.1:1234/v1/chat/completions"  

def query_llama(prompt):
    """Envia un prompt a LM Studio y recibe la respuesta."""
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 256, 
        # Es posible agregar otros parámetros de generación aca (temperature, top_p, etc.)
    }
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con LM Studio: {e}")
        return None
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error al procesar la respuesta de LM Studio: {e}")
        return None

def responder_pregunta(pregunta):
    """Busca documentos relevantes y luego consulta a Llama."""
    query_emb = get_embeddings(pregunta)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=3  # Obtener algunos documentos relevantes como contexto
    )

    if results["documents"]:
        contexto = "\n\n".join(results["documents"][0])
        prompt = f"Basándote en la siguiente información:\n\n{contexto}\n\nResponde a la siguiente pregunta: {pregunta}" # Juntar el contexto obtenido y la pregunta
        respuesta = query_llama(prompt)
        return respuesta
    else:
        return "No se encontraron documentos relevantes para responder a tu pregunta."

# --- Ejemplo de uso ---
if __name__ == "__main__":
    crear_base_datos() 

    while True:
        pregunta_usuario = input("Pregúntame algo sobre los documentos (o escribe 'salir'): ")
        print(f"👩 Pregunta: {pregunta_usuario}")
        if pregunta_usuario.lower() == 'salir':
            break

        respuesta_ia = responder_pregunta(pregunta_usuario)
        if respuesta_ia:
            print(f"🤖 Respuesta de la IA: {respuesta_ia}")

✅ Base de datos creada con éxito.
👩 Pregunta: ¿Por qué Cenicienta no pudo ir al baile inicialmente?
🤖 Respuesta de la IA: Cenicienta no pudo ir al baile inicialmente porque su madrastra y hermanastras malvadas le ordenaron que se quedara en casa. Estaban interesadas en impresionar al príncipe, por lo que prepararon lujosos vestidos y joyas para asistir al baile, dejando a Cenicienta atrás.
👩 Pregunta: ¿Qué ocurre cuando llega la medianoche en el baile?
🤖 Respuesta de la IA: Cuando llega la medianoche en el baile, Cenicienta debe regresar a su casa, ya que el hechizo se rompe y ella vuelve a ser una simple sirvienta. Si no lo hace, continuará siendo mágicamente atrapada y congelada en un estado de pérdida de la forma humana.
👩 Pregunta: ¿Qué hizo Maléfica al no ser invitada a la fiesta?
🤖 Respuesta de la IA: Maléfica fue ofendida cuando el rey y la reina olvidaron invitarla a la celebración de nacimiento de Aurora. Al no ser invitada, ella lanzó una maldición contra Aurora, diciendo que